In [13]:
import pandas as pd
import os
import sys
import torch
import re
import json
import math
import os
import glob
import os
import numpy as np
from torchvision.transforms import GaussianBlur
from torchvision import transforms
import torch.nn.functional as F
import torchvision.transforms as T
import cv2

from PIL import Image, ImageDraw, ImageFont, ImageFilter
current_dir = os.getcwd()
from pathlib import Path
# 현재 작업 경로 가져오기
current_dir = Path.cwd()
# 상위 폴더 가져오기
parent_dir = current_dir.parent
# sys.path에 추가 (문자열로 변환 필요)
sys.path.append(str(parent_dir))
print(f"Added to sys.path: {parent_dir}")
# 경로 설정이 완료된 후 import 해야 합니다.
from VLM_model_dot_relative import QwenVLModel, MetricsTracker
from file_managing import (
    load_selected_samples,
    get_actual_path,
    get_gt_path,
)
from config import AGD20K_PATH

Added to sys.path: /home/bongo/porter_notebook/research/qwen3


In [6]:
def load_ground_truth(gt_path):
    """
    Load and process ground truth image
    Args:
        gt_path (str): Path to the ground truth image
    Returns:
        torch.Tensor: Processed ground truth tensor normalized to [0, 1]
    """
    try:
        # Load the ground truth image
        gt_img = Image.open(gt_path)
        
        # Convert to grayscale if image is RGB
        if gt_img.mode == 'RGB':
            gt_img = gt_img.convert('L')
        
        # Convert to tensor
        gt_tensor = transforms.ToTensor()(gt_img).squeeze(0)
        
        # Normalize to [0, 1]
        if gt_tensor.max() > 0:
            gt_tensor = (gt_tensor - gt_tensor.min()) / (gt_tensor.max() - gt_tensor.min())
        
        return gt_tensor
        
    except Exception as e:
        print(f"⚠️ Failed to load ground truth image: {str(e)}")
        return None

def create_heatmap_from_dots_v2(image_size, dots):
    """
    Create a heatmap from dot coordinates using Gaussian kernels with dynamic sigma.
    Args:
        image_size (tuple): Size of the image (height, width)
        dots (list): List of dot coordinates [x, y]
    Returns:
        torch.Tensor: Heatmap tensor
    """
    height, width = image_size

    # Dynamic sigma based on image dimensions (simple linear scaling)
    base_size = 640  # Reference size
    base_sigma = 60
    scale_factor = ((height + width) / 2) / base_size
    sigma = int( base_sigma * scale_factor)
    heatmap = torch.zeros((height, width))
    for dot in dots:
        # Convert coordinates to integers
        x, y = map(int, dot)
        # Ensure coordinates are within image bounds
        x = max(0, min(x, width-1))
        y = max(0, min(y, height-1))
        # Create coordinate grids for the entire image
        y_grid, x_grid = torch.meshgrid(
            torch.arange(height, dtype=torch.float32),
            torch.arange(width, dtype=torch.float32),
            indexing='ij'
        )
        # Calculate Gaussian values centered at the dot
        gaussian = torch.exp(
            -((x_grid - x)**2 + (y_grid - y)**2) / (2 * sigma**2)
        )
        # Add to heatmap
        heatmap += gaussian
    # Normalize heatmap
    if heatmap.max() > 0:
        heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-10)
    return heatmap

def draw_dots_on_image( image_path, dots, gt_path, action, exo_path=None, exo_type=None, output_path=None):
    """
    Draw dots and create heatmap, save results side by side with GT
    Args:
        image_path (str): Path to the ego image
        dots (list): List of dot coordinates [x, y]
        gt_path (str): Path to the ground truth image
        action (str): Action name for the filename
        exo_path (str, optional): Path to the exo image (if provided, creates 3x2 layout)
        exo_type (str, optional): Type of exo image ('random' or 'selected')
        output_path (str, optional): Path to save the result image
    Returns:
        str: Path to the saved image
        torch.Tensor: Generated heatmap for metric calculation
    """
    # Load the ego image
    ego_img = Image.open(image_path)
    if exo_path is not None:
        exo_file_name = os.path.basename(exo_path)
    width, height = ego_img.size
    
    # Load exo image if provided
    exo_img = None
    if exo_path:
        exo_img = Image.open(exo_path)
    
    # Create heatmap from dots
    heatmap_tensor = create_heatmap_from_dots_v2((height, width), dots)
    
    # Convert heatmap to RGB image
    heatmap_img = transforms.ToPILImage()(heatmap_tensor.unsqueeze(0).repeat(3, 1, 1))
    
    # Create a copy for dot drawing
    dot_img = draw_dots_on_single_image(ego_img, dots, color='red', radius=15)
    
    # Determine layout based on image aspect ratio
    aspect_ratio = width / height
    
    # For very wide images (aspect ratio > 2), adjust font size based on width
    if aspect_ratio > 2:
        font_size = min(50, width // 12)  # Larger font for wide images
        header_height = 110  # Increased header height
        spacing = 30  # Normal spacing
    elif aspect_ratio > 1.5:  # For moderately wide images
        font_size = min(55, width // 10)  # Larger font for moderately wide images
        header_height = 120  # Increased header height
        spacing = 35  # Slightly increased spacing
    else:
        font_size = max(60, width // 8)  # Largest font size for normal images
        header_height = 130  # Normal header height
        spacing = 40  # Normal spacing
    
    # Create a new image with 3x2 layout
    combined_width = width * 3
    combined_height = height * 2 + header_height * 2 + spacing * 3 + 40  # Dynamic height
    combined_img = Image.new('RGB', (combined_width, combined_height), 'white')
    
    # Try to load fonts (size proportional to image width and aspect ratio)
    try:
        # Try to load a font that supports Korean
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", font_size)
    except:
        try:
            # Fallback to DejaVu font
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", font_size)
        except:
            # Last resort: default font
            font = ImageFont.load_default()

    # Get file names
    ego_filename = os.path.basename(image_path)
    gt_filename = os.path.basename(gt_path) if gt_path else "No GT"
    
    # Draw file names and titles for 3x2 layout
    draw = ImageDraw.Draw(combined_img)
    
    # Configure headers based on whether exo image is provided
    if exo_img:
        exo_filename = os.path.basename(exo_path)
        # Top row headers for exo version
        top_headers = [
            ("Ego", ego_filename),
            ("Exo", exo_filename),
            ("", "")  # Empty space
        ]
    else:
        # Top row headers for ego only version
        top_headers = [
            ("Original", ego_filename),
            ("", ""),  # Empty space
            ("", "")   # Empty space
        ]
    
    # Bottom row headers (same for both versions)
    bottom_headers = [
        ("Dots", action+"_"+ego_filename),
        ("Heatmap", action+"_"+ ego_filename),
        ("GT", action+"_"+gt_filename)
    ]
    
    # Draw top row headers with background
    for idx, (title, filename) in enumerate(top_headers):
        if title:  # Only draw if not empty
            section_width = width
            section_x = idx * section_width
            
            # Draw white background for text area
            draw.rectangle([section_x, 0, section_x + section_width, header_height], fill='white', outline='lightgray')
            
            # Draw title
            title_width = draw.textlength(title, font=font)
            title_x = section_x + (section_width - title_width) // 2
            draw.text((title_x, 5), title, fill='black', font=font)
            
            # Draw filename (truncate if too long)
            max_filename_width = section_width - 20
            filename_truncated = filename
            while draw.textlength(filename_truncated + "...", font=font) > max_filename_width and len(filename_truncated) > 0:
                filename_truncated = filename_truncated[:-1]
            if filename_truncated != filename:
                filename_truncated += "..."
            
            filename_width = draw.textlength(filename_truncated, font=font)
            filename_x = section_x + (section_width - filename_width) // 2
            draw.text((filename_x, header_height // 2 + 5), filename_truncated, fill='black', font=font)
    
    # Draw bottom row headers with background
    for idx, (title, filename) in enumerate(bottom_headers):
        section_width = width
        section_x = idx * section_width
        section_y = height + header_height + spacing  # Position below first row
        
        # Draw white background for text area
        draw.rectangle([section_x, section_y - 10, section_x + section_width, section_y + header_height], fill='white', outline='lightgray')
        
        # Draw title
        title_width = draw.textlength(title, font=font)
        title_x = section_x + (section_width - title_width) // 2
        draw.text((title_x, section_y), title, fill='black', font=font)
        
        # Draw filename (truncate if too long)
        max_filename_width = section_width - 20
        filename_truncated = filename
        while draw.textlength(filename_truncated + "...", font=font) > max_filename_width and len(filename_truncated) > 0:
            filename_truncated = filename_truncated[:-1]
        if filename_truncated != filename:
            filename_truncated += "..."
        
        filename_width = draw.textlength(filename_truncated, font=font)
        filename_x = section_x + (section_width - filename_width) // 2
        draw.text((filename_x, section_y + header_height // 2), filename_truncated, fill='black', font=font)
    
    # Paste images in 3x2 layout
    # Top row: Ego image and optionally Exo image
    top_image_y = header_height + spacing
    combined_img.paste(ego_img, (0, top_image_y))  # Ego
    if exo_img:
        combined_img.paste(exo_img, (width, top_image_y))  # Exo
    
    # Bottom row: Dots, Heatmap, GT
    bottom_image_y = height + header_height * 2 + spacing * 2
    combined_img.paste(dot_img, (0, bottom_image_y))  # Image with dots
    combined_img.paste(heatmap_img, (width, bottom_image_y))  # Heatmap
    
    # Add GT image and calculate metrics
    gt_map = load_ground_truth(gt_path)
    metrics_text = "No GT provided"
    
    if gt_map is not None:
        if isinstance(gt_map, torch.Tensor):
            gt_img = transforms.ToPILImage()(gt_map.unsqueeze(0).repeat(3, 1, 1))
        else:
            gt_map_tensor = torch.tensor(gt_map)
            gt_img = transforms.ToPILImage()(gt_map_tensor.unsqueeze(0).repeat(3, 1, 1))
        combined_img.paste(gt_img, (width * 2, bottom_image_y))  # GT heatmap
        
        # Calculate metrics
        metrics = calculate_metrics(heatmap_tensor, gt_map)
        metrics_text = f"KLD: {metrics['KLD']:.4f} | SIM: {metrics['SIM']:.4f} | NSS: {metrics['NSS']:.4f}"
    else:
        # If no GT provided, create blank white image
        blank_img = Image.new('RGB', (width, height), 'white')
        combined_img.paste(blank_img, (width * 2, bottom_image_y))
        metrics_text = "ERRRR"
    
    # Draw metrics text at the bottom with background
    text_width = draw.textlength(metrics_text, font=font)
    text_x = (combined_width - text_width) // 2
    text_y = bottom_image_y + height + spacing
    
    # Draw white background for metrics text
    padding = 10
    draw.rectangle([text_x - padding, text_y - padding, 
                    text_x + text_width + padding, text_y + font_size + padding], 
                    fill='white', outline='gray')
    
    draw.text((text_x, text_y), metrics_text, fill='black', font=font)
    
    # Create res_images directory if it doesn't exist
    script_dir = os.path.dirname(os.path.abspath(current_dir ))
    if exo_type is None:
        res_dir = os.path.join(script_dir, f'dot_images')
        os.makedirs(res_dir, exist_ok=True)
        os.makedirs(os.path.join(res_dir, "with_exo"), exist_ok=True)
        os.makedirs(os.path.join(res_dir, "only_ego"), exist_ok=True)
    else:
        res_dir = os.path.join(script_dir, f'dot_images_{exo_type}')
        os.makedirs(res_dir, exist_ok=True)
        os.makedirs(os.path.join(res_dir, "with_exo"), exist_ok=True)
        os.makedirs(os.path.join(res_dir, f"{exo_type}"), exist_ok=True)
        os.makedirs(os.path.join(res_dir, "only_ego"), exist_ok=True)            
        
    # Generate output path if not provided
    if output_path is None:
        base_name = os.path.splitext(ego_filename)[0]
        ext = os.path.splitext(ego_filename)[1]
        if exo_img and exo_type:
            # Format: skis_002829_jump_exo_random.jpg or skis_002829_jump_exo_selected.jpg
            output_filename = f"{base_name}_{action}_exo_{exo_type}{ext}"
            output_path = os.path.join(res_dir, f"with_exo/{output_filename}")
        elif exo_img:
            # Fallback if exo_type not specified
            output_filename = f"{base_name}_{action}_exo_{exo_file_name}"
            output_path = os.path.join(res_dir, f"with_exo/{output_filename}")
        elif exo_type is not None:
            output_filename = f"{base_name}_{action}_exo_{exo_type}{ext}"
            output_path = os.path.join(res_dir, f"{exo_type}/{output_filename}")
        else:
            # Format: skis_002829_jump.jpg
            output_filename = f"{base_name}_{action}{ext}"
            output_path = os.path.join(res_dir, f"only_ego/{output_filename}")
    
    # Save the combined image
    combined_img.save(output_path)
    # print(f"✅ Saved comparison image with heatmap and GT: {output_path}")
    
    return output_path, heatmap_tensor

def draw_dots_on_single_image( image, dots, color='red', radius=15):
    """
    Draw dots on an image
    Args:
        image (PIL.Image): Image to draw on
        dots (list): List of dot coordinates [x, y]
        color (str): Color of the dots
        radius (int): Radius of the dots
    Returns:
        PIL.Image: Image with dots drawn
    """
    img_copy = image.copy()
    draw = ImageDraw.Draw(img_copy)
    
    for dot in dots:
        x, y = map(int, dot)
        # Draw circle
        draw.ellipse([x-radius, y-radius, x+radius, y+radius], 
                    fill=color, outline=color)
    
    return img_copy


def calculate_metrics(pred_heatmap, gt_map):
    """
    Calculate comparison metrics between predicted heatmap and GT (following original metric.py)
    Args:
        pred_heatmap (torch.Tensor): Predicted heatmap
        gt_map (torch.Tensor): Ground truth map
    Returns:
        dict: Dictionary containing KLD, SIM, and NSS metrics
    """
    # Ensure inputs are proper tensors
    if not isinstance(pred_heatmap, torch.Tensor):
        pred_heatmap = torch.tensor(pred_heatmap)
    if not isinstance(gt_map, torch.Tensor):
        gt_map = torch.tensor(gt_map)
    
    # Flatten tensors and add batch dimension for compatibility
    pred = pred_heatmap.flatten().float().unsqueeze(0)  # [1, H*W]
    gt = gt_map.flatten().float().unsqueeze(0)          # [1, H*W]
    
    eps = 1e-10
    
    # Calculate KLD following original implementation
    # Normalize to probability distributions
    pred_norm = pred / pred.sum(dim=1, keepdim=True)
    gt_norm = gt / gt.sum(dim=1, keepdim=True)
    pred_norm += eps
    kld = F.kl_div(pred_norm.log(), gt_norm, reduction="batchmean").item()
    
    # Calculate SIM following original implementation
    pred_sim = pred / pred.sum(dim=1, keepdim=True)
    gt_sim = gt / gt.sum(dim=1, keepdim=True)
    sim = torch.minimum(pred_sim, gt_sim).sum().item() / len(pred_sim)
    
    # Calculate NSS following original implementation
    # First normalize by max values
    pred_nss = pred / pred.max(dim=1, keepdim=True).values
    gt_nss = gt / gt.max(dim=1, keepdim=True).values
    
    # Calculate z-score for prediction
    std = pred_nss.std(dim=1, keepdim=True)
    u = pred_nss.mean(dim=1, keepdim=True)
    smap = (pred_nss - u) / (std + eps)
    
    # Create fixation map from GT
    fixation_map = (gt_nss - torch.min(gt_nss, dim=1, keepdim=True).values) / (
        torch.max(gt_nss, dim=1, keepdim=True).values - torch.min(gt_nss, dim=1, keepdim=True).values + eps)
    fixation_map = (fixation_map >= 0.1).float()
    
    # Calculate NSS
    nss_values = smap * fixation_map
    nss = nss_values.sum(dim=1) / (fixation_map.sum(dim=1) + eps)
    nss = nss.mean().item()
    
    return {
        'KLD': kld,
        'SIM': sim,
        'NSS': nss
    }

In [48]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def recover_heatmap_data(image_path, target_size=(31, 31)):
    # 1. 이미지 로드 (BGR 형태)
    img_bgr = cv2.imread(image_path)
    
    if img_bgr is None:
        print(f"❌ 파일을 찾을 수 없습니다: {image_path}")
        return None

    # 2. 빨간색 네모 부분 찾기 (마스크 생성)
    # BGR 기준 빨간색 범위: Blue/Green은 낮고, Red는 높은 영역
    lower_red = np.array([0, 0, 150])   # 진한 빨강 ~ 
    upper_red = np.array([100, 100, 255]) # 밝은 빨강
    
    red_mask = cv2.inRange(img_bgr, lower_red, upper_red)
    
    # 마스크를 살짝 넓혀서 네모의 경계선까지 확실하게 포함
    kernel = np.ones((3,3), np.uint8)
    red_mask = cv2.dilate(red_mask, kernel, iterations=1)

    # 3. 인페인팅 (Inpainting): 빨간 부분을 주변 색으로 덮어쓰기
    # 주변 픽셀값을 참조하여 자연스럽게 메꿉니다.
    img_clean = cv2.inpaint(img_bgr, red_mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

    # 4. 31x31 크기로 리사이즈 (Downsampling)
    # 이미지가 클 경우, 정보를 압축하는 INTER_AREA 방식이 가장 좋습니다.
    img_resized = cv2.resize(img_clean, target_size, interpolation=cv2.INTER_AREA)

    # 5. 점수(Score)로 변환
    # BGR -> Grayscale 변환
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    
    # 'gray_r' (0=흰색, 1=검정색)을 사용했다고 가정하고 역산
    # 검정색(0)에 가까울수록 높은 점수(1.0)
    # 흰색(255)에 가까울수록 낮은 점수(0.0)
    heatmap_scores = 1.0 - (img_gray.astype(float) / 255.0)
    # 상단 1행 (0번째 인덱스)
    heatmap_scores[0, :] = 0.0
    
    # 하단 1행 (마지막 인덱스)
    heatmap_scores[-1, :] = 0.0
        
    # 좌측 1열 (0번째 인덱스)
    heatmap_scores[:, 0] = 0.0
    
    # 우측 1열 (마지막 인덱스)
    heatmap_scores[:, -1] = 0.0
    return heatmap_scores


In [7]:
df_fin = pd.read_pickle('/home/bongo/porter_notebook/research/qwen3/APM_dot_verifying/test_verify_qwen3_32b.pkl')
df_fin.loc[df_fin['final_dot'].apply(lambda x : len(x))==0,'final_dot']= df_fin.loc[df_fin['final_dot'].apply(lambda x : len(x))==0,'dots']
df_fin

,action,object,filename,dots,veri_result,veri_reason,final_dot
0,jump,skis,skis_002829.jpg,"[[280, 490], [720, 490], [280, 900], [720, 900]]","[Fail, Fail, Fail, Fail]","[The query point (280,490) is located on the t...","[[280, 490], [720, 490], [280, 900], [720, 900]]"
1,jump,skateboard,skateboard_002387.jpg,"[[500, 350], [350, 650], [650, 650]]","[Pass, Pass, Fail]","[The query point (500,350) lies on the central...","[[500, 350], [350, 650]]"
2,jump,surfboard,surfboard_000658.jpg,"[[200, 200], [500, 500], [800, 800]]","[Fail, Fail, Fail]","[The query point (200,200) is located in the w...","[[200, 200], [500, 500], [800, 800]]"
3,jump,snowboard,snowboard_001704.jpg,"[[250, 200], [500, 500], [750, 200]]","[Pass, Fail, Fail]","[The query point (250,200) lies on the top sur...","[[250, 200]]"
4,peel,carrot,carrot_003707.jpg,"[[224, 341], [455, 385], [715, 441]]","[Pass, Pass, Pass]","[The query point (224,341) lies on the surface...","[[224, 341], [455, 385], [715, 441]]"
...,...,...,...,...,...,...,...
116,eat,apple,apple_001541.jpg,"[[500, 460], [350, 400], [700, 450]]","[Pass, Pass, Pass]","[The query point (500,460) lies on the surface...","[[500, 460], [350, 400], [700, 450]]"
117,swing,baseball_bat,baseball_bat_001882.jpg,"[[500, 500], [200, 750], [700, 250]]","[Fail, Fail, Pass]","[The point (500,500) is located on the barrel ...","[[700, 250]]"
118,swing,tennis_racket,tennis_racket_003066.jpg,"[[500, 300], [500, 650], [500, 900]]","[Pass, Fail, Fail]","[The query point (500,300) lies within the str...","[[500, 300]]"
119,swing,golf_clubs,golf_clubs_001992.jpg,"[[175, 48], [165, 495], [135, 950]]","[Fail, Fail, Fail]","[The query point (175,48) is located near the ...","[[175, 48], [165, 495], [135, 950]]"


In [ ]:

# --- 실행 ---
file_path = "/root/qwen3_AG/APM_dot_verifying/top_attention_heads/brush_with/toothbrush/toothbrush_001764.jpg_495_115_rank01_L01_H10_score_1.0000.png"

restored_data = recover_heatmap_data(file_path, target_size=(31, 31))

if restored_data is not None:
    print(f"✅ 데이터 복원 완료")
    print(f" - Shape: {restored_data.shape}")
    print(f" - Min: {restored_data.min():.4f}, Max: {restored_data.max():.4f}")
    
    # 시각화 확인
    plt.figure(figsize=(5, 5))
    plt.imshow(restored_data, cmap='jet')
    plt.title("Restored 31x31 Heatmap")
    plt.colorbar()
    plt.show()

디렉토리 생성 시도: ./attention_output
디렉토리 생성 시도: ./attention_output/pairs
디렉토리 생성 시도: ./attention_output/each_dots

✅ 모든 출력 디렉토리 확인 및 생성 완료.


In [ ]:
import os

OUTPUT_DIR = "./attention_output"
OUTPUT_DIR_PAIRS = os.path.join(OUTPUT_DIR, "pairs")
OUTPUT_DIR_EACH_DOTS = os.path.join(OUTPUT_DIR, "each_dots")

# 1. 상위 디렉토리 생성
print(f"디렉토리 생성 시도: {OUTPUT_DIR}")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2. 하위 디렉토리 생성
print(f"디렉토리 생성 시도: {OUTPUT_DIR_PAIRS}")
os.makedirs(OUTPUT_DIR_PAIRS, exist_ok=True)

print(f"디렉토리 생성 시도: {OUTPUT_DIR_EACH_DOTS}")
os.makedirs(OUTPUT_DIR_EACH_DOTS, exist_ok=True)

print("\n✅ 모든 출력 디렉토리 확인 및 생성 완료.")

# Get total number of samples
total_samples = len(df_fin)
metrics_tracker_ego = MetricsTracker(name="only_ego")
# Process each sample
print(f"Processing {total_samples} samples...")
print("=" * 50)    
for idx, row in df_fin.iterrows():
    object_name = row['object']
    action = row['action']
    filename = row['filename']
    dot_list =  row['dots']
    image_path = f"{AGD20K_PATH}/Seen/testset/egocentric/{action}/{object_name}/{filename}"
    original_image = cv2.imread(image_path)
    original_h, original_w, _ = original_image.shape
    attention_file_path = f"/home/bongo/porter_notebook/research/qwen3/APM_dot_verifying/QWEN3_32B_top_attention_heads/{action}/{object_name}/{filename}_495_115_rank01_L01_H10_score_1.0000.png"

    gt_path = get_gt_path(image_path) 
    image_name = image_path.split('/')[-1]

    print(f"Action : {action}, Object : {object_name} image_name : {image_name}")

    final_dots = row['final_dot']
    total_heatmap_sum = np.zeros((original_h, original_w), dtype=np.float32)
    for x, y in final_dots:

        matching_files = []
        base_pattern = (
            f"/home/bongo/porter_notebook/research/qwen3/APM_dot_verifying/"
            f"QWEN3_32B_top_attention_heads/{action}/{object_name}/{filename}_{x}_{y}_rank"
        )
        # 결과 출력
        # 가져오고 싶은 rank의 최대 숫자 (예: 5로 설정하면 rank01, rank02, ..., rank05만 가져옵니다.)
        use_k = 10
        # rank01 부터 rank{k} 까지 순회하며 정확한 파일 이름을 검색합니다.
        for i in range(1, use_k + 1):
            # i를 두 자리 숫자로 포매팅합니다 (예: 1 -> "01", 10 -> "10")
            rank_number = f"{i:02d}"
            
            # 전체 파일 경로를 생성합니다.
            file_pattern = f"{base_pattern}{rank_number}*.png"
            found_files = glob.glob(file_pattern)
            matching_files.extend(found_files)
        if len(matching_files) != use_k:
            raise Exception(f"에러!! 발견된 파일 목록 (총 {len(matching_files)}개) - {use_k}개가 왜 안되지??:")
        
        for i, file_path in enumerate(matching_files):
            # 3. 히트맵 데이터 복구
            patch_data = recover_heatmap_data(file_path)
            # if i == 0:
            #     # 첫 번째 배열은 최종 배열의 초기값으로 설정합니다.
            #     final_array = patch_data
            # else:
            #     final_array = final_array * patch_data 
            patch_data_upsampled = cv2.resize(
                                        patch_data, 
                                        (original_w, original_h), 
                                        interpolation=cv2.INTER_LINEAR
                                    )

            # 3. [변경] 업샘플링된 맵끼리 곱하기 (Hadamard Product)
            if i == 0:
                final_array = patch_data_upsampled
            else:
                final_array = final_array + patch_data_upsampled

        if final_array is None:
            raise Exception(f"에러!! final_array 가 없음!!")
        # 2. 업샘플링 (Resize)
        # final_array를 0~1 사이 값으로 정규화 (최대값이 1이 되도록)
        heatmap_normalized = final_array / final_array.max()


        # cv2.resize를 사용하여 원본 이미지 크기(original_w, original_h)로 확대
        # Interpolation: INTER_LINEAR 또는 INTER_CUBIC 사용 권장
        # heatmap_upsampled = cv2.resize(
        #     heatmap_normalized, 
        #     (original_w, original_h), 
        #     interpolation=cv2.INTER_LINEAR
        # )
        total_heatmap_sum += heatmap_normalized # heatmap_upsampled
        # print(f"  > Upsampled heatmap size: {heatmap_upsampled.shape}")
        # 3. 마스크 생성 및 오버레이

        # 히트맵을 0-255 범위의 uint8 타입으로 변환
        # (히트맵이 0~1 사이의 부동소수점 값이라고 가정)
        # heatmap_255 = np.uint8(255 * heatmap_upsampled)
        heatmap_255 = np.uint8(255 * heatmap_normalized)

        # 히트맵을 컬러맵으로 변환 (예: JET 컬러맵)
        # 1채널 히트맵(회색조)을 3채널 컬러 이미지로 변환
        heatmap_color = cv2.applyColorMap(heatmap_255, cv2.COLORMAP_JET)

        # 이미지 오버레이 (가중치 합산)
        # heatmap_color는 0-255, image도 0-255입니다.
        # cv2.addWeighted(src1, alpha, src2, beta, gamma)
        # 최종 이미지 = src1 * alpha + src2 * beta + gamma
        # 여기서는 원본 이미지(image) 60% + 히트맵(heatmap_color) 40%로 혼합
        alpha = 0.6  # 원본 이미지 투명도
        beta = 0.4   # 히트맵 투명도
        
        # 이미지는 BGR 포맷으로 가정 (cv2 기본)
        overlayed_image = cv2.addWeighted(image, alpha, heatmap_color, beta, 0)

        # 시각화된 결과 저장 (옵션)
        dots_output_filename = f"{object_name}_{action}_{filename.replace('.jpg', '')}_{x}_{y}_att_overlay.png"
        dots_output_path = os.path.join(OUTPUT_DIR,'each_dots', dots_output_filename)
        cv2.imwrite(dots_output_path, overlayed_image)
        # print(f"  > ✅ 시각화 결과 저장: {output_path}")

    final_sum_normalized = total_heatmap_sum / total_heatmap_sum.max()
    # 2. 컬러맵 적용
    final_sum_255 = np.uint8(255 * final_sum_normalized)
    final_sum_color = cv2.applyColorMap(final_sum_255, cv2.COLORMAP_JET)

    # 3. 원본 이미지와 오버레이
    # (image는 BGR 포맷의 원본 이미지)
    alpha = 0.6  # 원본 이미지 투명도
    beta = 0.4   # 히트맵 투명도
    final_overlayed_image = cv2.addWeighted(image, alpha, final_sum_color, beta, 0)

    # 4. 파일 저장
    pairs_output_filename = f"{object_name}_{action}_{filename.replace('.jpg', '')}_att_overlay.png"
    pair_output_path = os.path.join(OUTPUT_DIR,'pairs', pairs_output_filename)
    cv2.imwrite(pair_output_path, final_overlayed_image)
    print(f"  > ✅ 통합 히트맵 저장: {pair_output_path}")


디렉토리 생성 시도: ./attention_output
디렉토리 생성 시도: ./attention_output/pairs
디렉토리 생성 시도: ./attention_output/each_dots

✅ 모든 출력 디렉토리 확인 및 생성 완료.
Processing 121 samples...
Action : jump, Object : skis image_name : skis_002829.jpg
  > ✅ 통합 히트맵 저장: ./attention_output/pairs/skis_jump_skis_002829_att_overlay.png
Action : jump, Object : skateboard image_name : skateboard_002387.jpg
  > ✅ 통합 히트맵 저장: ./attention_output/pairs/skateboard_jump_skateboard_002387_att_overlay.png
Action : jump, Object : surfboard image_name : surfboard_000658.jpg
  > ✅ 통합 히트맵 저장: ./attention_output/pairs/surfboard_jump_surfboard_000658_att_overlay.png
Action : jump, Object : snowboard image_name : snowboard_001704.jpg
  > ✅ 통합 히트맵 저장: ./attention_output/pairs/snowboard_jump_snowboard_001704_att_overlay.png
Action : peel, Object : carrot image_name : carrot_003707.jpg
  > ✅ 통합 히트맵 저장: ./attention_output/pairs/carrot_peel_carrot_003707_att_overlay.png
Action : peel, Object : orange image_name : orange_001193.jpg
  > ✅ 통합 히트맵 저장

In [23]:
np.shape(recover_heatmap_data(file_path))

(31, 31)

In [21]:
file_path

'/home/bongo/porter_notebook/research/qwen3/APM_dot_verifying/QWEN3_32B_top_attention_heads/swing/badminton_racket/badminton_racket_003649.jpg_250_750_rank02_L11_H43_score_1.0000.png'

In [ ]:
# Get total number of samples
total_samples = len(df_fin)
metrics_tracker_ego = MetricsTracker(name="only_ego")
# Process each sample
print(f"Processing {total_samples} samples...")
print("=" * 50)    
for idx, row in df_fin.iterrows():
    object_name = row['object']
    action = row['action']
    filename = row['filename']
    dot_list =  row['dots']
    image_path = f"{AGD20K_PATH}/Seen/testset/egocentric/{action}/{object_name}/{filename}"
    gt_path = get_gt_path(image_path) 
    image_name = image_path.split('/')[-1]
    print(f"Action : {action}, Object : {object_name} image_name : {image_name}")
    final_dots = row['final_dot']
    print(f"parsed dots!!! : {final_dots}")

    raw_image = Image.open(image_path).convert("RGB")
    orig_width, orig_height = raw_image.size

    dots = [
                [int(x * (orig_width / 1000)), int(y * (orig_height / 1000))] 
                for x, y in final_dots
            ]
    print(f"restored_dots!!! : {dots}")

    dot_image_path, heatmap_tensor = draw_dots_on_image(image_path, dots, gt_path, action)

    # Save heatmap image
    script_dir = os.path.dirname(os.path.abspath(current_dir))
    res_dir = os.path.join(script_dir, f'dot_images', 'heatmaps')
    os.makedirs(res_dir, exist_ok=True)
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    ext = os.path.splitext(image_path)[1]
    heatmap_filename = f"{base_name}_{action}_heatmap{ext}"
    heatmap_path = os.path.join(res_dir, heatmap_filename)
    
    # Convert heatmap tensor to image and save
    heatmap_img = transforms.ToPILImage()(heatmap_tensor.unsqueeze(0).repeat(3, 1, 1))
    heatmap_img.save(heatmap_path)

    # Save dot image separately for validation  
    dot_res_dir = os.path.join(script_dir, f'dot_images', 'dots_only')
    os.makedirs(dot_res_dir, exist_ok=True)
    dot_only_filename = f"{base_name}_{action}_dots{ext}"
    dot_only_path = os.path.join(dot_res_dir, dot_only_filename)
        
    # Create dot image (ego image with dots)
    ego_img = Image.open(image_path)
    dot_only_img = draw_dots_on_single_image(ego_img, dots, color='red', radius=15)
    dot_only_img.save(dot_only_path)
    
    # Calculate metrics if GT is available
    metrics = None
    gt_map = load_ground_truth(gt_path)
    if gt_map is not None and len(dots) > 0:
        metrics = calculate_metrics(heatmap_tensor, gt_map)
    
    dot_images_res =  {
        'dots': dots,
        'dot_image_path': dot_image_path,
        'dot_only_image_path': dot_only_path,
        'heatmap_image_path': heatmap_path,
        'heatmap_tensor': heatmap_tensor,
        'metrics': metrics,
    }

    metrics_ego = dot_images_res['metrics']

    if metrics_ego:
        # Update and print metrics
        metrics_tracker_ego.update(metrics_ego)
        metrics_tracker_ego.print_metrics(metrics_ego, image_path.split('/')[-1])